In [1]:
import cv2

In [2]:
# Função utilizada para redimensionar a imagem de entrada
def redim(img, largura):
    # Calcula a proporção da imagem
    alt = int(img.shape[0] / img.shape[1] * largura)
    # Redimensiona a imagem
    img = cv2.resize(img, (largura, alt), interpolation=cv2.INTER_AREA)
    # Retorna a imagem redimensionada
    return img


In [3]:
def detecta_rostos_em_video(df, camera, largura_redimensionada_saida, fps_saida, caminho_saida):
    # Inicializa contadores
    contador_total = 0
    contador_acertos = 0

    # Configurações de redimensionamento
    altura_redimensionada = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT) *
                                largura_redimensionada_saida / camera.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Configurações do vídeo de saída
    nome_video_saida = f'{caminho_saida}/deteccao_rostos_saida.mp4'
    codec = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para vídeo MP4

    # Inicializar o objeto VideoWriter
    video_saida = cv2.VideoWriter(
        nome_video_saida, codec, fps_saida, (largura_redimensionada_saida, altura_redimensionada))

    while True:
        # Ler o próximo frame
        (sucesso, frame) = camera.read()
        if not sucesso:
            break
        # Redimensionar o frame de entrada
        frame = redim(frame, largura_redimensionada_saida)
        # Converter o frame para tons de cinza
        frame_pb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detectar rostos no frame
        faces = df.detectMultiScale(frame_pb, scaleFactor=1.1, minNeighbors=3, minSize=(
            20, 20), flags=cv2.CASCADE_SCALE_IMAGE)
        # Desenhar retângulos nos rostos detectados
        frame_temp = frame.copy()
        for (x, y, lar, alt) in faces:
            cv2.rectangle(frame_temp, (x, y),
                          (x + lar, y + alt), (0, 255, 255), 2)

        # Atualiza os contadores
        contador_total += 1
        if len(faces) > 0:
            contador_acertos += 1

        # Escrever o frame processado no vídeo de saída
        video_saida.write(frame_temp)

        # Interrompe o loop quando pressionar a tecla "s"
        if cv2.waitKey(1) & 0xFF == ord("s"):
            break

    # Imprime os resultados
    print(f"Acertos totais: {contador_acertos}")
    print(f"Total de frames: {contador_total}")

    # Calcula e imprime a porcentagem de acertos
    porcentagem_acertos = (contador_acertos / contador_total) * 100
    print(
        f'Porcentagem de frames com pelo menos um rosto detectado: {porcentagem_acertos}%')

    # Libera os objetos
    camera.release()
    # Fechar o objeto VideoWriter
    video_saida.release()
    # Fechar todas as janelas
    cv2.destroyAllWindows()


In [5]:
df = cv2.CascadeClassifier(
    './opencv/data/haarcascades/haarcascade_frontalface_alt.xml')
camera = cv2.VideoCapture(
    './videos/video_frontal_face.mp4')
caminho_saida = './videos'
largura_redimensionada = 1080
fps_saida = 30

detecta_rostos_em_video(
    df, camera, largura_redimensionada, fps_saida, caminho_saida)


[aac @ 0x132fadbb0] Input buffer exhausted before END element found


Acertos totais: 404
Total de frames: 511
Porcentagem de frames com pelo menos um rosto detectado: 79.06066536203522%
